# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 21 2023 2:55PM,259686,10,MSP219797,"Methapharm, Inc.",Released
1,Mar 21 2023 2:55PM,259686,10,MSP219798,"Methapharm, Inc.",Released
2,Mar 21 2023 2:53PM,259685,10,MSP219792,"Methapharm, Inc.",Released
3,Mar 21 2023 2:53PM,259685,10,MSP219794,"Methapharm, Inc.",Released
4,Mar 21 2023 2:53PM,259685,10,MSP219795,"Methapharm, Inc.",Released
5,Mar 21 2023 2:53PM,259685,10,MSP219796,"Methapharm, Inc.",Released
6,Mar 21 2023 2:41PM,259684,10,MSP219769,Methapharm-G,Released
7,Mar 21 2023 2:39PM,259683,20,ATR-I000111,"HVL, LLC dba Atrium Innovations",Released
8,Mar 21 2023 2:36PM,259682,19,ADV80014279,"AdvaGen Pharma, Ltd",Released
9,Mar 21 2023 2:36PM,259682,19,ADV80014275,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
23,259682,Released,5
24,259683,Released,1
25,259684,Released,1
26,259685,Released,4
27,259686,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
259682,NaN,NaN,5.0
259683,NaN,NaN,1.0
259684,NaN,NaN,1.0
259685,NaN,NaN,4.0
259686,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
259596,0.0,1.0,0.0
259606,42.0,26.0,5.0
259610,0.0,1.0,1.0
259626,0.0,0.0,1.0
259629,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
259596,0,1,0
259606,42,26,5
259610,0,1,1
259626,0,0,1
259629,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,259596,0,1,0
1,259606,42,26,5
2,259610,0,1,1
3,259626,0,0,1
4,259629,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,259596,,1,
1,259606,42,26,5
2,259610,,1,1
3,259626,,,1
4,259629,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 21 2023 2:55PM,259686,10,"Methapharm, Inc."
2,Mar 21 2023 2:53PM,259685,10,"Methapharm, Inc."
6,Mar 21 2023 2:41PM,259684,10,Methapharm-G
7,Mar 21 2023 2:39PM,259683,20,"HVL, LLC dba Atrium Innovations"
8,Mar 21 2023 2:36PM,259682,19,"AdvaGen Pharma, Ltd"
13,Mar 21 2023 2:31PM,259680,10,Emerginnova
14,Mar 21 2023 2:31PM,259679,10,Emerginnova
15,Mar 21 2023 2:22PM,259668,10,Bio-PRF
23,Mar 21 2023 2:15PM,259665,19,"GUSA Granules USA, Inc."
24,Mar 21 2023 2:13PM,259664,10,"Snap Medical Industries, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 21 2023 2:55PM,259686,10,"Methapharm, Inc.",,,2
1,Mar 21 2023 2:53PM,259685,10,"Methapharm, Inc.",,,4
2,Mar 21 2023 2:41PM,259684,10,Methapharm-G,,,1
3,Mar 21 2023 2:39PM,259683,20,"HVL, LLC dba Atrium Innovations",,,1
4,Mar 21 2023 2:36PM,259682,19,"AdvaGen Pharma, Ltd",,,5
5,Mar 21 2023 2:31PM,259680,10,Emerginnova,,,1
6,Mar 21 2023 2:31PM,259679,10,Emerginnova,,,1
7,Mar 21 2023 2:22PM,259668,10,Bio-PRF,,,8
8,Mar 21 2023 2:15PM,259665,19,"GUSA Granules USA, Inc.",,,1
9,Mar 21 2023 2:13PM,259664,10,"Snap Medical Industries, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 21 2023 2:55PM,259686,10,"Methapharm, Inc.",2,,
1,Mar 21 2023 2:53PM,259685,10,"Methapharm, Inc.",4,,
2,Mar 21 2023 2:41PM,259684,10,Methapharm-G,1,,
3,Mar 21 2023 2:39PM,259683,20,"HVL, LLC dba Atrium Innovations",1,,
4,Mar 21 2023 2:36PM,259682,19,"AdvaGen Pharma, Ltd",5,,
5,Mar 21 2023 2:31PM,259680,10,Emerginnova,1,,
6,Mar 21 2023 2:31PM,259679,10,Emerginnova,1,,
7,Mar 21 2023 2:22PM,259668,10,Bio-PRF,8,,
8,Mar 21 2023 2:15PM,259665,19,"GUSA Granules USA, Inc.",1,,
9,Mar 21 2023 2:13PM,259664,10,"Snap Medical Industries, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 21 2023 2:55PM,259686,10,"Methapharm, Inc.",2,,
1,Mar 21 2023 2:53PM,259685,10,"Methapharm, Inc.",4,,
2,Mar 21 2023 2:41PM,259684,10,Methapharm-G,1,,
3,Mar 21 2023 2:39PM,259683,20,"HVL, LLC dba Atrium Innovations",1,,
4,Mar 21 2023 2:36PM,259682,19,"AdvaGen Pharma, Ltd",5,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 21 2023 2:55PM,259686,10,"Methapharm, Inc.",2.0,NaN,NaN
1,Mar 21 2023 2:53PM,259685,10,"Methapharm, Inc.",4.0,NaN,NaN
2,Mar 21 2023 2:41PM,259684,10,Methapharm-G,1.0,NaN,NaN
3,Mar 21 2023 2:39PM,259683,20,"HVL, LLC dba Atrium Innovations",1.0,NaN,NaN
4,Mar 21 2023 2:36PM,259682,19,"AdvaGen Pharma, Ltd",5.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 21 2023 2:55PM,259686,10,"Methapharm, Inc.",2.0,0.0,0.0
1,Mar 21 2023 2:53PM,259685,10,"Methapharm, Inc.",4.0,0.0,0.0
2,Mar 21 2023 2:41PM,259684,10,Methapharm-G,1.0,0.0,0.0
3,Mar 21 2023 2:39PM,259683,20,"HVL, LLC dba Atrium Innovations",1.0,0.0,0.0
4,Mar 21 2023 2:36PM,259682,19,"AdvaGen Pharma, Ltd",5.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3115902,31.0,2.0,0.0
15,259659,2.0,18.0,0.0
19,2596462,16.0,28.0,42.0
20,259683,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3115902,31.0,2.0,0.0
1,15,259659,2.0,18.0,0.0
2,19,2596462,16.0,28.0,42.0
3,20,259683,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,31.0,2.0,0.0
1,15,2.0,18.0,0.0
2,19,16.0,28.0,42.0
3,20,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,31.0
1,15,Released,2.0
2,19,Released,16.0
3,20,Released,1.0
4,10,Executing,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20
Status,,,,
Completed,0.0,0.0,42.0,0.0
Executing,2.0,18.0,28.0,0.0
Released,31.0,2.0,16.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20
0,Completed,0.0,0.0,42.0,0.0
1,Executing,2.0,18.0,28.0,0.0
2,Released,31.0,2.0,16.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20
0,Completed,0.0,0.0,42.0,0.0
1,Executing,2.0,18.0,28.0,0.0
2,Released,31.0,2.0,16.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()